CREATING ENTIRE GRAPH WITH ATTRIBUTES FROM TRAINING DATA CSV

### IMPORT STATEMENTS

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from io import StringIO
import random 
import pickle
import os
from dotenv import load_dotenv

load_dotenv()

True

### GRAPH CREATION

In [4]:
# Read the CSV file into a DataFrame
train_df = pd.read_csv('train_data.csv')
print(train_df.head())


                                        Event ID  Event Date Event Type  \
0  20231231-1075-cb07b4c0139b_THREATEN_territory  2023-12-31   THREATEN   
1            20240102-2349-3b800104f953_SANCTION  2024-01-02   SANCTION   
2             20240118-9689-3e8dc91f684d_REQUEST  2024-01-18    REQUEST   
3            20240121-9346-c2f7b7626d1a_THREATEN  2024-01-21   THREATEN   
4            20240121-9943-c2f7b7626d1a_THREATEN  2024-01-21   THREATEN   

  Event Mode  Event Intensity          Quad Code               Contexts  \
0  territory             -4.0    VERBAL CONFLICT  military | terrorism    
1        NaN             -6.0  MATERIAL CONFLICT             terrorism    
2        NaN              0.0    VERBAL CONFLICT  military | terrorism    
3        NaN             -3.5    VERBAL CONFLICT             terrorism    
4        NaN             -3.5    VERBAL CONFLICT             terrorism    

           Actor Name Actor Country  Actor COW  ... GeoNames ID Raw Placename  \
0  Benjamin Netan

In [6]:
G = nx.Graph()
# Function to add nodes and edges
def add_event_to_graph(row):

    # Date
    date = row['Event Date']
    G.add_node(date)

    # Event attributes
    event_id = row['Event ID']
    event_date = row['Event Date']
    event_type = row['Event Type']
    event_intensity = row['Event Intensity']
    quad_code = row['Quad Code']
    contexts = row['Contexts']
    
    #Other attributes

    #Event Type Node 
    G.add_node(event_type)

    # Create an event node
    G.add_node(event_id, event_date=event_date, event_type=event_type,event_intensity=event_intensity,quad_code=quad_code,contexts=contexts)

    # Add actor nodes with attributes
    actor = row['Actor Name']
    if actor == '':
        actor=row['Actor Country']
    G.add_node(actor, country=row['Actor Country'],cow=row['Actor COW'], primary_sector=row['Primary Actor Sector'],sectors=row['Actor Sectors'],title=row['Actor Title'],id=row['Wikipedia Actor ID'], actor_type='actor') 

    # Add recipient nodes with attributes
    recipient = row['Recipient Name'] 
    G.add_node(recipient, country=row['Recipient Country'], cow=row['Recipient COW'], primary_sector=row['Primary Recipient Sector'],sectors=row['Recipient Sectors'],title=row['Recipient Title'],id=row['Wikipedia Recipient ID'],actor_type='recipient')

    #Location node
    place = row['Raw Placename']
    G.add_node(place,city=row['City'],district=row['District'],province=row['Province'],latitude=row['Latitude'],longitude=row['Longitude'],geoid=row['GeoNames ID'],raw=row['Raw Placename'])

    # Add information source node
    info_source = row['Source']
    G.add_node(info_source, publication_date=row['Publication Date'],feature=row['Feature Type'], date=row['Publication Date'],people=row['Story People'],organization=row['Story Organizations'],location=row['Story Locations'],language=row['Language'], type='source')

    # Add edges between nodes
    G.add_edge(event_id,event_type,relation='type')
    G.add_edge(event_id,date,relation='date')
    G.add_edge(event_id, actor, relation='actor')
    G.add_edge(event_id, recipient, relation='recipient')
    G.add_edge(event_id, place, relation='location')
    G.add_edge(event_id, info_source, relation='source')

# Iterate over each row to build the graph
for index, row in train_df.iterrows():
    add_event_to_graph(row)

In [9]:
# Save the graph using pickle
with open('new_graph.pkl', 'wb') as f:
    pickle.dump(G, f)

In [10]:
with open('new_graph.pkl', 'rb') as f:
    G = pickle.load(f)